# Скрапим и парсим данные

**Благодарим за разработку парсера - Александра Коробова (ЦТИИ ГПБ).**

Машинное обучение без данных, как говорят, что компьютер без электричества. Любая, какая бы то ни было модель, не покажет стоящего результата, если данные будут плохого качества. А чем больше данных, тем больше вероятность, что из них можно вытащить что-либо полезное.

Поэтому важно уметь (и иметь возможность) доставать новые данные. Например, **скрапить** — извлекать информацию с сайтов.

Это можно сделать на языке Python, например, с помощью фреймворков `requests` или `selenium`. Первая библиотека позволяет «общаться» с веб-сервисом за счёт HTTP-запросов, а вторая моделирует действия реального пользователя.

Однако за счёт одного запроса мы не сможем вытащить нужную нам информацию в удобном (главное, читаемом) виде, потому что результатом запроса может являться как html-код, так и текст типа json — необходимо обработать эти данные. Иначе говоря, **распарсить**, то есть автоматически обработать данные, поступаемые в сложно интерпретируемом формате, во что-то осознанное и систематизированное.

Ниже посмотрим, как можно извлекать данные с сайтов с помощью запросов `requests`, распарсить их, собрать полученные данные в датасет и сохранить всё в удобной табличке. 

Здесь приведён сразу *сырой пример* программы, позволяющего парсить объявления о продаже квартир на сайте **Циан**. Некоторые части кода будут более подробно рассмотрены далее.

In [ ]:
import os
import csv
import json
import requests
import cfscrape
import traceback
from tqdm import tqdm
from time import sleep


PAUSE_TIME = 5 # Увеличиваем интервалы отправки запросов — борьба с капчей
CSV_NUMBER = 'test' # Постфикс названия создаваемой таблицы
CSV_PATH = os.path.normpath(os.path.join(os.getcwd(), 'csv')) # Создаём папку 'csv' для записи создаваемых таблиц
if not os.path.exists(CSV_PATH): # ...если такой не существовало ранее
    os.mkdir(CSV_PATH)
    print(f'Folder {CSV_PATH} has been created!')


# Словарь некоторых городов с номерами, объявления по которым можно искать на Циан
regions = {
    'msk': 1, # Москва
    'spb': 2, # Санкт–Петербург
    'ekb': 4743, # Екатеринбург
    'nsk': 4897, # Новосибирск
    'kzn': 4777, # Казань
    'nng': 4885, # Нижний Новгород
}

# Названия столбцов (header) будущей таблицы,
# которые связываются с отобранными признаками в create_table()
dataset = [
            [ 
                'region',
                'address',
                'price',
                'total_area',
                'kitchen_area', 
                'living_area',
                'rooms_count',
                'floor', 
                'floors_number',
                'build_date',
                'isСomplete',
                'complitation_year',
                'house_material',
                'parking',
                'decoration',
                'balcony',
                'longitude', 
                'latitude',
                'passenger_elevator', 
                'cargo_elevator', 
                'metro', 
                'metro_distance', 
                'metro_transport',
                'district',
                'is_apartments',
                'is_auction'
            ]
        ]


# Функция для обработки пропусков и булевых значений
def add_attr(attr):
    if isinstance(attr, bool):
        return int(attr)
        
    return attr if attr is not None else 'empty'


# Функция для создания экземпляра класса запросов
def get_session():
    # Передаваемые параметры для экземпляра (как получить — туториал ниже)
    
#     headers = {
#         'authority': 'api.cian.ru',
#         'accept': '*/*',
#         'accept-language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
#         'content-type': 'application/json',
#         'cookie': '_CIAN_GK=8ad76c13-5d73-434f-a106-85e76bc875c2; _gcl_au=1.1.603512102.1709739126; login_mro_popup=1; tmr_lvid=fd128fd61fa70340de24133346787875; tmr_lvidTS=1709739125862; sopr_utm=%7B%22utm_source%22%3A+%22direct%22%2C+%22utm_medium%22%3A+%22None%22%7D; uxfb_usertype=searcher; uxs_uid=b08c1360-dbce-11ee-a235-dbd798ce88c8; _ym_uid=1709739127926818209; _ym_d=1709739127; _gid=GA1.2.1679918364.1709739127; adrcid=AqVeMksRrLuFDGWuGh5eHAA; afUserId=376096fb-0dea-4523-8b7d-b3a5bcc5e04b-p; AF_SYNC=1709739130749; cookie_agreement_accepted=1; _gpVisits={"isFirstVisitDomain":true,"idContainer":"1000252B"}; _ym_isad=2; session_region_id=1; session_main_town_region_id=1; my_home_tooltip_key=1; __cf_bm=xfCAkbpX9ppffyTsT4WWIO79gqKeezq_65L69zg7FyQ-1709836575-1.0.1.1-ip4jO9ORUzxF4nT29EAvddfrHOBsiCV2ESdzMtheO1WvBvkazkD8.KIfbht2NzeD7mvp1XhDrXTetG6eSz_v4Q; anti_bot="2|1:0|10:1709837263|8:anti_bot|44:eyJyZW1vdGVfaXAiOiAiMjEyLjE2NC42NS4yMDIifQ==|03fa3fbfbf2d89f2dcd6fa5bded6206894d6c04032651ca823bc2c358f01ed20"; sopr_session=443ce283b49c41e9; _ga=GA1.2.890488983.1709739127; _dc_gtm_UA-30374201-1=1; _ym_visorc=b; _ga_3369S417EL=GS1.1.1709837259.11.1.1709837276.43.0.0',
#         'origin': 'https://www.cian.ru',
#         'referer': 'https://www.cian.ru/',
#         'sec-ch-ua': '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
#         'sec-ch-ua-mobile': '?0',
#         'sec-ch-ua-platform': '"macOS"',
#         'sec-fetch-dest': 'empty',
#         'sec-fetch-mode': 'cors',
#         'sec-fetch-site': 'same-site',
#         'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
#     }

    session = requests.Session()
    session.headers = headers
    return cfscrape.create_scraper(sess=session) # cfscrape — обход защиты от ботов Cloudflare


# Записываем всё в файл формата .csv
def recording_table():
    try:
        with open(os.path.join(CSV_PATH, f'data_{CSV_NUMBER}.csv'), mode='w', newline='', encoding='utf-8') as file:
                writer = csv.writer(file)
                for row in dataset:
                    writer.writerow(row)

        print(f'The dataset is written in file "data_{CSV_NUMBER}.csv"')
        return
            
    except Exception as error:
        print('Recording error!\n', traceback.format_exc())
        sleep(PAUSE_TIME)
        return
    

# Получаем формат json (питоновский dict) из нашего запроса Response        
def get_json(session, region_name, cur_page):
    # Параметры, которые отображаются в URL-запросе
    # https://www.cian.ru/cat.php/?deal_type=sale&engine_version=2&offer_type=flat&region=1&
    # room1=1&room2=1&room3=1&room4=1&room5=1&room6=1
    json_data = {
            'jsonQuery': {
                '_type': 'flatsale',
                'engine_version': {
                    'type': 'term',
                    'value': 2,
                },
                'region': {
                    'type': 'terms',
                    'value': [
                        regions[region_name],
                    ],
                },
                'room': {
                    'type': 'terms',
                    'value': [
                        1,
                        2,
                        3,
                        4,
                        5,
                        6,
                    ],
                },
                'page': {
                    'type': 'term',
                    'value': cur_page,
                },
                # Можно задавать дополнительные атрибуты фильтрации объявлений
                
                # 'price': {
                #     'type': 'range',
                #     'value': {
                #         'gte': min_price,
                #         'lte': max_price,
                #     },
                # },
            },
        }
        
    # Получаем запрос с заданными параметрами
    # Возвращаемое значение — bytes
    try:
        response = session.post('https://api.cian.ru/search-offers/v2/search-offers-desktop/',
                                json=json_data)

    except:
        return f'oops! Error {response.status_code}'

    # Получаем формат .json
    if (
        response.status_code != 204 and 
        response.headers["content-type"].strip().startswith("application/json")
    ):
        try:
            return response.json()
        except ValueError:
            return f'oops! ValueError!'


def create_table(region_name='msk', start_page=1, end_page=55, number_of_samples=100):
    # В Циан выдаются страницы в диапазоне [1, 54]
    if start_page < 1:
        start_page = 1
    if end_page > 55:
        end_page = 55
    
    session = get_session()

    cnt_samples = 0
    for cur_page in tqdm(range(start_page, end_page)): # tqdm — выводим прогресс выполнения цикла
        if cnt_samples >= number_of_samples:
            break
            
        data = get_json(session, region_name, cur_page)
        if data is None:
            print('oops! Captcha!')
            return
        if isinstance(data, str):
            print(data)
            continue
        
        # Отбираем из большого словаря то, что нам нужно (можно и больше — смотри data)
        for item in data['data']['offersSerialized']:
            cur_item = [
                    region_name,
                    add_attr(item["geo"]["userInput"]),
                    add_attr(item['bargainTerms']['priceRur']),
                    add_attr(item.get('totalArea')),
                    add_attr(item.get('kitchenArea')),
                    add_attr(item.get('livingArea')),
                    add_attr(item.get('roomsCount')),
                    add_attr(item.get('floorNumber')),
                    add_attr(item['building'].get('floorsCount')),
                    add_attr(item['building'].get('buildYear')),
                    add_attr(item['building']['deadline']['isComplete'] if item['building'].get('deadline') is not None else None),
                    add_attr(item['building']['deadline']['year'] if item['building'].get('deadline') is not None else None),
                    add_attr(item['building'].get('materialType')),
                    add_attr(item['building']['parking']['type'] if item['building'].get('parking') is not None else None),
                    add_attr(item.get('decoration')),
                    add_attr(item.get('balconiesCount')),
                    add_attr(item['geo']['coordinates']['lng']),
                    add_attr(item['geo']['coordinates']['lat']),
                    add_attr(item['building'].get('passengerLiftsCount')),
                    add_attr(item['building'].get('cargoLiftsCount')),
                    add_attr(','.join([str(x['name']) for x in item['geo']['undergrounds']if x is not None])),
                    add_attr(','.join([str(x['time']) for x in item['geo']['undergrounds'] if x is not None])),
                    add_attr(','.join([str(x['transportType']) for x in item['geo']['undergrounds'] if x is not None])),
                    add_attr(','.join([str(x['name']) for x in item['geo']['districts'] if x is not None])),
                    add_attr(item.get('isApartments')), 
                    add_attr(item.get('isAuction'))
                ]
            
            if cur_item not in dataset:
                dataset.append(cur_item)
                cnt_samples += 1
            else:
                continue

            if cnt_samples >= number_of_samples:
                break

        print(f'{cnt_samples} / {number_of_samples} | page: {cur_page}')
        sleep(PAUSE_TIME)
                
    recording_table()
    return


create_table()

**Методы программы:**

* `add_attr` — для обработки пропусков и булевых значений; чтобы первые в табличке прописывались словом 'empty', а вторые были в бинарном виде;
* `get_session` — метод, создающий экземпляр класса `requests.Session()`; отличие от обычного `requests` в том, что он позволяет сохранить определённые параметры запроса для сайта (помогает в борьбе с капчей за счёт использования cookie; что такое словари cookies и headers, а также туториал по тому, как их достать, — дальше;
* `recording_table` — метод для записи получившейся таблицы на диск;
* `get_json` — метод, которыq отправляет запросы на сайта Циана, обрабатывает полученные результаты и возвращает их в формате .json (в питоне хранится в виде типа `dict`);
* `create_table` — основная функция — метод для обработки получающихся в `get_json` словарей и выделение из них необходимых нам признаков.

## Чуть более подробно о параметрах

Для начала мы в методе `get_session` создаем объект класса, с помощью которого будем передавать запросы на сайт Циана. В нём мы передаём следующие параметры через словарь `headers`. Его можно сгенерировать с помощью сторонних утилит! Ровно так же, как и `json_data` из метода `get_json`.

Дело в том, что практически все сайты-агрегаторы — помимо кода html — хранят у себя в коде страницы уже готовый json, который можно отыскать для простоты генераций запросов, чтобы избежать проблем с парсингом html. :)

В Циане это обстоит немного иначе: если нажать на их сайте кнопку «Найти» и выбрать нужные параметры, то сайт отправит запрос (аналогично тому, что мы отправляем на сам сайт) базе данных, с которой вернётся результат уже в формате `.json` — мы можем сделать это напрямую, заодно получив 

Более конкретно:

1. Заходим, например, на Московский (можно и на любой другой, но могут возникнуть трудности) сайт Циана ([ссылка](https://www.cian.ru/)).
2. Выбираем фильтры, по которым мы хотим формировать будущую табличку — это будущий словарь `json_data` (например, в примере выше выбрана продажа квартир ('flatsale') в Москве с комнатами от 1 до 6; при этом в запросе мы просматриваем страничку `cur_page`).
3. Открываем панель разработчика (`Ctrl + Shift + I`, `Cmd + option + I` и т.п.) и переходим во вкладку «Сеть» («Network»).
4. Нажима на поиск на сайте.
5. Ищем запрос `search-offers-dekstop`; если на него нажать, то будет что-то вроде: `{data: {,…}, status: "ok"}`.
6. Нажимаем правой кнопкой и копируем как `cURL`.
7. Заходим на сайт [cURL Converter](https://curlconverter.com) и вставляем скопированное в пункте 6.
8. Получаем необходимые (и готовые) словари `headers` и `json_data` — вы великолепны!

**Замечение про капчу!**

Использование `requests.Session()`, `cfscrape` и `sleep(PAUSE_TIME)` не ограничивают от того, что ваши запросы сайт расценит как активность робота и вывалит вам капчу вместо ответа — во избежание этого нужно писать более сложный код. Чтобы это поправить, нужно обновить параметр `cookie` словаря `json_data` перед очередным запуском кода. Как получить новое значение словаря? *Проделать пункты 1.—8., пройдя предварительно вручную проверку на то, что под вашим ip скрывается на робот*.

**Замечение про замечание и всё остальное!**

Всё изложенное выше претендует лишь на пример того, как можно собирать новые данные в интернете, и вовсе не выдаётся за абсолютно верный путь. Цель этого — передать идею и продемонстрировать функционал. Но всё, так или иначе, в ваших руках — пробуйте, ошибайтесь, набирайтесь опыта и, в конце концов, становитесь крутыми специалистами!